In [181]:
import numpy
bert_embs = numpy.load("../../../data/embedding_data/st_short_desc_summaries.npz")
bert_embs = [bert_embs[f] for f in bert_embs.files]

In [182]:
tender_refs = open("../../../data/embedding_data/tender_references.txt", "r")
content = ""
for line in tender_refs:
    content = line
    break

In [183]:
content = content.replace("[", "")
content = content.replace("'", "")
content = content.replace(",", "")
refs = content.split(" ")
print(len(refs))
print(len(bert_embs))

20619
20619


In [184]:
from sklearn.cluster import KMeans
from sklearn import preprocessing
kmeans = KMeans(n_clusters = 2000, n_init = 1).fit_predict(preprocessing.normalize(bert_embs))

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f408ee180e0>
Traceback (most recent call last):
  File "/home/ucc/maxichat/miniconda3/envs/capstone/lib/python3.11/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/ucc/maxichat/miniconda3/envs/capstone/lib/python3.11/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ucc/maxichat/miniconda3/envs/capstone/lib/python3.11/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
                   ^^^^^^^^^^^^^^^^^^
  File "/home/ucc/maxichat/miniconda3/envs/capstone/lib/python3.11/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().s

In [236]:
import pandas as pd
tenders_structured_path = r"~/Capstone/UWACapstoneG2/data/UpdatedAgainTenders.xlsx"

tenders_structured = pd.read_excel(tenders_structured_path)
tenders_structured = tenders_structured[["Reference Number", "Contract Title", "Description"]].dropna(subset=["Reference Number"]).drop_duplicates()

In [237]:
len(tenders_structured)

20625

In [239]:
len(kmeans)

20619

In [259]:
clusters = pd.DataFrame({"Reference Number": refs, "cluster": kmeans})
print(len(clusters))

20619


In [260]:
clusters = pd.merge(clusters, tenders_structured, on = "Reference Number")
len(clusters)

20373

In [262]:
from bs4 import BeautifulSoup
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    cleaned = soup.get_text().replace('\xa0', ' ')
    cleaned = ' '.join(cleaned.split())
    return cleaned

docs = []
for index, row in clusters.iterrows():
    title = row["Contract Title"]
    desc = remove_html_tags(row["Description"])
    clusters.at[index, "Description"] = desc
    docs.append(title + ". " + desc)

In [263]:
clusters.loc[clusters["cluster"] == 5 , ["Reference Number", "Contract Title", "Description"]] 

,Reference Number,Contract Title,Description
7571,DMIRS21780,Provision of Tengraph Replacement and eMiTS De...,This RFQ requires the services of J2EE develop...
7804,DMIRS350320,Provision of J2EE Services to Support the Deve...,Provision of J2EE Services to Support the Deve...


In [300]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
tender_tfidf = TfidfVectorizer(stop_words="english",
                               min_df=1, max_df = 0.3)
tender_tfidf.fit(docs)

TfidfVectorizer(max_df=0.3, stop_words='english')

In [301]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(
    n_components=1,
    max_iter=20,
    learning_method="batch",
)

In [302]:
def display_topic(n_cluster):
    subset_docs = [docs[i] for i in list(np.where(clusters["cluster"] == n_cluster)[0])]
    tfidf = tender_tfidf.transform(subset_docs)
    lda = LatentDirichletAllocation(
        n_components=1,
        max_iter=20,
        learning_method="batch",
    )
    lda_fitted = lda.fit(tfidf)
    topics = tender_tfidf.get_feature_names_out()[lda_fitted.components_.argsort()]
    topics = list(topics[0][-5:])
    print(topics)
    return clusters.loc[clusters["cluster"] == n_cluster, ["Reference Number", "Contract Title", "Description"]]

In [322]:
display_topic(45)

['school', 'naturaliste', 'college', 'cuaelc2012', 'electricity']


,Reference Number,Contract Title,Description
1941,ED230341,Electricity - Cape Naturaliste College (CUAELC...,Electricity - Cape Naturaliste College (CUAELC...
1986,ED230234,Electricity - Koorana Primary School (CUAELC2012),Electricity - Koorana Primary School (CUAELC2012)
2182,ed220455,Electricity - Como Secondary College,Electricity - Como Secondary College
2273,ED210330,Electricity - Warriapendi Primary School (CUAE...,Electricity - Warriapendi Primary School (CUAE...
2340,ED210203,Electricity - Wandina Primay School (CUAELC2012),Electricity - Wandina Primay School (CUAELC2012)
2374,ED210165,Electricity - Cape Naturaliste College (CUAELC...,Electricity - Cape Naturaliste College (CUAELC...
2486,ED200387,Electricity - Capel Primary School (CUAELC2012),Electricity - Capel Primary School (CUAELC2012)
2505,ED200338,Electricity - Kyilla Primary School (CUAELC2012),Electricity - Kyilla Primary School (CUAELC2012)
2527,ED200314,Electricity - Dianella Secondary College (CUAE...,Electricity - Dianella Secondary College (CUAE...
2582,ED200256,Electricity - Pinjarra Primary School (CUAELC2...,Electricity - Pinjarra Primary School (CUAELC2...
